In [1]:
import sqlite3
import pandas as pd

In [37]:
df = pd.read_csv("ola_rides_cleaned.csv")

In [44]:
df.head()

,booking_id,booking_status,customer_id,vehicle_type,pickup_location,drop_location,v_tat,c_tat,booking_value,payment_method,ride_distance,driver_ratings,customer_rating,vehicle_images,ride_day,ride_month,ride_hour,cancelled_by,booking_datetime,completed_flag
0,CNR2940424040,Completed,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,158,CASH,13,4.1,4.0,https://cdn-icons-png.flaticon.com/128/9983/99...,Thursday,7,22,NaN,2024-07-25 22:20:00,1
1,CNR2982357879,Completed,CID270156,Prime Suv,Sahakar Nagar,Varthur,238.0,130.0,386,UPI,40,4.2,4.8,https://cdn-icons-png.flaticon.com/128/9983/99...,Tuesday,7,19,NaN,2024-07-30 19:59:00,1
2,CNR1797421769,Completed,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,822,CREDIT CARD,45,4.0,3.0,https://cdn-icons-png.flaticon.com/128/3202/32...,Tuesday,7,9,NaN,2024-07-02 09:02:00,1
3,CNR8787177882,Completed,CID802429,Mini,Kadugodi,Vijayanagar,231.0,90.0,173,UPI,41,3.4,4.6,https://cdn-icons-png.flaticon.com/128/3202/32...,Saturday,7,4,NaN,2024-07-13 04:42:00,1
4,CNR3612067560,Completed,CID476071,Bike,Tumkur Road,Whitefield,133.0,40.0,140,CASH,49,3.2,4.5,https://cdn-icons-png.flaticon.com/128/9983/99...,Tuesday,7,9,NaN,2024-07-23 09:51:00,1


In [7]:
# MEMORY IN DATABASE

In [45]:
conn = sqlite3.connect(":memory:")

In [8]:
# DATAFRAME INTO SQL TABLE

In [46]:
df.to_sql("ola_rides", conn, index=False, if_exists="replace")
print(" SQL environment ready. Table name: ola_rides")

 SQL environment ready. Table name: ola_rides


In [9]:
#Q1. Retrieve all successful bookings

In [47]:
query1="""SELECT *
FROM ola_rides
WHERE booking_status = 'Completed';"""

In [48]:
df_q1 = pd.read_sql_query(query1, conn)
df_q1.head()

,booking_id,booking_status,customer_id,vehicle_type,pickup_location,drop_location,v_tat,c_tat,booking_value,payment_method,ride_distance,driver_ratings,customer_rating,vehicle_images,ride_day,ride_month,ride_hour,cancelled_by,booking_datetime,completed_flag
0,CNR2940424040,Completed,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,158,CASH,13,4.1,4.0,https://cdn-icons-png.flaticon.com/128/9983/99...,Thursday,7,22,None,2024-07-25 22:20:00,1
1,CNR2982357879,Completed,CID270156,Prime Suv,Sahakar Nagar,Varthur,238.0,130.0,386,UPI,40,4.2,4.8,https://cdn-icons-png.flaticon.com/128/9983/99...,Tuesday,7,19,None,2024-07-30 19:59:00,1
2,CNR1797421769,Completed,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,822,CREDIT CARD,45,4.0,3.0,https://cdn-icons-png.flaticon.com/128/3202/32...,Tuesday,7,9,None,2024-07-02 09:02:00,1
3,CNR8787177882,Completed,CID802429,Mini,Kadugodi,Vijayanagar,231.0,90.0,173,UPI,41,3.4,4.6,https://cdn-icons-png.flaticon.com/128/3202/32...,Saturday,7,4,None,2024-07-13 04:42:00,1
4,CNR3612067560,Completed,CID476071,Bike,Tumkur Road,Whitefield,133.0,40.0,140,CASH,49,3.2,4.5,https://cdn-icons-png.flaticon.com/128/9983/99...,Tuesday,7,9,None,2024-07-23 09:51:00,1


In [13]:
#Q2. Find the average ride distance for each vehicle type

In [50]:
query2="""SELECT 
    vehicle_type,
    AVG(ride_distance) AS avg_ride_distance
FROM ola_rides
WHERE ride_distance > 0
GROUP BY vehicle_type;"""

In [52]:
df_q2 = pd.read_sql_query(query2, conn)
df_q2

,vehicle_type,avg_ride_distance
0,Auto,10.040689
1,Bike,24.933873
2,Ebike,25.146187
3,Mini,24.978198
4,Prime Plus,25.034270
5,Prime Sedan,25.006397
6,Prime Suv,24.883059


In [53]:
(df_q2['avg_ride_distance'] <= 0).sum()

np.int64(0)

In [54]:
df.groupby('vehicle_type')['ride_distance'].mean()

vehicle_type
Auto           10.040689
Bike           24.933873
Ebike          25.146187
Mini           24.978198
Prime Plus     25.034270
Prime Sedan    25.006397
Prime Suv      24.883059
Name: ride_distance, dtype: float64

In [15]:
#Q3. Get the total number of cancelled rides by customers

In [16]:
query3="""SELECT 
    COUNT(*) AS cancelled_by_customers
FROM ola_rides
WHERE booking_status = 'Cancelled'
  AND cancelled_by = 'Customer';"""

In [55]:
df_q3 = pd.read_sql_query(query3, conn)
df_q3

,cancelled_by_customers
0,0


In [56]:
df[(df['booking_status']=='Cancelled') &
   (df['cancelled_by']=='Customer')].shape[0]

0

In [17]:
#Q4. List the top 5 customers who booked the highest number of rides

In [18]:
query4="""SELECT 
    customer_id,
    COUNT(*) AS total_rides
FROM ola_rides
GROUP BY customer_id
ORDER BY total_rides DESC
LIMIT 5;"""

In [57]:
df_q4 = pd.read_sql_query(query4, conn)
df_q4

,customer_id,total_rides
0,CID836942,4
1,CID635963,4
2,CID329193,4
3,CID288207,4
4,CID268274,4


In [19]:
#Q5. Get the number of rides cancelled by drivers due to personal and car-related issues

In [66]:
query5="""SELECT 
    cancellation_reason_driver,
    COUNT(*) AS total_cancellations
FROM ola_rides
WHERE booking_status = 'Cancelled'
  AND cancelled_by = 'Driver'
GROUP BY cancellation_reason_driver;"""

In [59]:
# no rides are cancelled so i have deleted the column 

In [21]:
#Q6. Find the maximum and minimum driver ratings for Prime Sedan bookings

In [61]:
query6="""SELECT 
    MAX(driver_ratings) AS max_driver_ratings,
    MIN(driver_ratings) AS min_driver_ratings
FROM ola_rides
WHERE vehicle_type = 'Prime Sedan'
  AND driver_ratings > 0;"""

In [62]:
pd.read_sql_query(query6, conn)

,max_driver_ratings,min_driver_ratings
0,5.0,3.0


In [23]:
#Q7. Retrieve all rides where payment was made using UPI

In [24]:
query7="""SELECT *
FROM ola_rides
WHERE payment_method = 'UPI';"""

In [63]:
df_q7 = pd.read_sql_query(query7, conn)
df_q7.shape

(25881, 20)

In [25]:
#Q8. Find the average customer rating per vehicle type

In [26]:
query8="""SELECT 
    vehicle_type,
    AVG(customer_rating) AS avg_customer_rating
FROM ola_rides
WHERE customer_rating > 0
GROUP BY vehicle_type;"""

In [64]:
pd.read_sql_query(query8, conn)

,vehicle_type,avg_customer_rating
0,Auto,3.998811
1,Bike,3.993376
2,Ebike,3.987854
3,Mini,3.997731
4,Prime Plus,4.009499
5,Prime Sedan,4.001589
6,Prime Suv,3.999378


In [27]:
#Q9. Calculate the total booking value of rides completed successfully

In [28]:
query9="""SELECT 
    SUM(booking_value) AS total_booking_value
FROM ola_rides
WHERE booking_status = 'Completed';"""

In [67]:
pd.read_sql_query(query9, conn)

,total_booking_value
0,35080467


In [68]:
df[df['booking_status']=='Completed']['booking_value'].sum()

np.int64(35080467)

In [29]:
#Q10. List all incomplete rides along with the reason

In [30]:
query10="""SELECT 
    booking_id,
    booking_status,
    cancellation_reason_customer,
    cancellation_reason_driver
FROM ola_rides
WHERE booking_status = 'Incomplete';"""

In [70]:
# there is no a single incomple ride and i have deleted te column 